<a href="https://colab.research.google.com/github/stephen-e-cox/whipPy/blob/main/whipPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [275]:
from google.colab import auth
auth.authenticate_user()

import re
from itertools import groupby
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import gspread
import gspread_dataframe as gd
from google.colab import auth
from google.auth import default

auth.authenticate_user()
creds, _ = default()

gc = gspread.authorize(creds)

In [276]:
## Edit this section for each new import

racedate = datetime.date(2023, 2, 25) ## date in year-month-day format
racedist = 4 ## distance in one of the below units (for named distances like marathon, use 1)
racedist_units = "miles" ## miles, km, m, marathon, half marathon
racefile_name = "2023_NYRR_Al_Gordon_4M"

In [287]:
## This is the master spreadsheet
wb = gc.open_by_url("https://docs.google.com/spreadsheets/d/1n_xmaQuNmj3JBfOXJEN8c4HZAOUPOjtPAM-fY4h4K54/edit#gid=0")

## This is the spreadsheet and worksheet containing current time standards
stds = gc.open_by_url("https://docs.google.com/spreadsheets/d/1G7a3kSUUD1TZRWZbeACTwQxQfW4yCdJqC5qytQ-EgdM/edit#gid=0")
qual_stds = stds.worksheet('2022_standards').get_all_values()

## This is the new race file to read. Drop it in the temporary files list in Colab. It will not be saved.
racefile = open(racefile_name + ".txt")

## This creates a new worksheet with the name of the racefile. If it already
## exists, it will be deleted!
try:
  ws = wb.add_worksheet(racefile_name, rows=1, cols=1)
except:
  ws = wb.worksheet(racefile_name)
  wb.del_worksheet(ws)
  ws = wb.add_worksheet(racefile_name, rows=1, cols=1)

## This creates a single archive of the summary spreadsheet, and deletes any
## existing one. This is just a light backup solution in case you screw up an
## import
sum_worksheet = wb.worksheet("Summary")
try:
  wb.duplicate_sheet(sum_worksheet.id, new_sheet_name="Summary_archive")
except:
  sum_arch_worksheet = wb.worksheet("Summary_archive")
  wb.del_worksheet(sum_arch_worksheet)
  wb.duplicate_sheet(sum_worksheet.id, new_sheet_name="Summary_archive")

In [277]:
## Process the race results into a friendly format

## Split it up by line first
racefile.seek(0)
data = racefile.read().split('\n')

## The format we get from NYRR page copy/paste is eight lines per entry
chunks = [data[x:x+8] for x in range(0, len(data), 8)]

racedata = pd.DataFrame(chunks, columns=["Name", "AgeLocBib", "Text1", "Time", "Text2", "Pace", "Text3", "Place"])

## Split Age/Location/Bib line
racedata[["Age", "LocBib"]] = racedata["AgeLocBib"].str.split(n=1, expand=True)
racedata[["Location", "Text4", "Bib"]] = racedata["LocBib"].str.rsplit(n=2, expand=True)
racedata['Gender'] = racedata['Age'].str[0]
racedata['Age'] = racedata['Age'].str[1:]

## We don't need labels like "Time" from the text file. We are also not using these
## to reshape because there is no "Name" or "Age/Location" label
racedata.drop(columns=["AgeLocBib", "LocBib", "Text1", "Text2", "Text3", "Text4"], inplace=True)

## Format numeric columns. There is probably a more elegant way to do this but it
## is tough to convince python/Pandas to treat time-formatted columns
## as length of time rather than timestamp or elapsed time since X
Time_formatted = pd.to_datetime(racedata["Time"], format = '%H:%M:%S')
Pace_formatted = pd.to_datetime(racedata["Pace"], format = '%M:%S')
racedata['Time_s'] = ((Time_formatted - Time_formatted.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
racedata['Pace_s'] = ((Pace_formatted - Pace_formatted.dt.normalize()) / pd.Timedelta('1 second')).astype(int)
racedata["Place"] = racedata["Place"].map(lambda s: s.replace(',',''))
racedata["Place"] = pd.to_numeric(racedata["Place"])
racedata["Age"] = pd.to_numeric(racedata["Age"])

In [283]:
## Load the qualifying time
stds_df = pd.DataFrame(qual_stds)
stds_df.columns = stds_df.iloc[0]
stds_df = stds_df.iloc[1:]

stds_df["Distance"] = pd.to_numeric(stds_df["Distance"])
stds_df["Hours"] = pd.to_numeric(stds_df["Hours"])
stds_df["Minutes"] = pd.to_numeric(stds_df["Minutes"])
stds_df["Seconds"] = pd.to_numeric(stds_df["Seconds"])

stds_df['Time_standard_s'] = stds_df['Hours']*3600 + stds_df['Minutes']*60 + stds_df['Seconds']

# try: 
dist_filter = stds_df['Distance'] == racedist
unit_filter = stds_df['Units'] == racedist_units

women_filter = stds_df['Gender'] == "W"
men_filter = stds_df['Gender'] == "M"
nonbinary_filter = stds_df['Gender'] == "X"

## If the standard is not present, it will be set to zero, which will result in 
## no runners achieving the standard, and an empty qualifying table

try:
  std_women = stds_df.loc[dist_filter & unit_filter & women_filter]['Time_standard_s'].item()
except:
  std_women = 0

try:
  std_men = stds_df.loc[dist_filter & unit_filter & men_filter]['Time_standard_s'].item()
except:
  std_men = 0

try:
  std_nonbinary = stds_df.loc[dist_filter & unit_filter & nonbinary_filter]['Time_standard_s'].item()
except:
  std_nonbinary = 0

In [288]:
## Search for qualifying times and top performances
racedata_women = racedata.query("Gender == 'W'")
racedata_men = racedata.query("Gender == 'M'")
racedata_nonbinary = racedata.query("Gender == 'X'")

top_women = racedata_women.nsmallest(10, columns="Time_s")
top_men = racedata_men.nsmallest(10, columns="Time_s")
top_nonbinary = racedata_nonbinary.nsmallest(10, columns="Time_s")

qualifying_women = racedata_women.query("Time_s <= {}".format(std_women))
qualifying_men = racedata_men.query("Time_s <= {}".format(std_men))
qualifying_nonbinary = racedata_nonbinary.query("Time_s <= {}".format(std_nonbinary))

In [289]:
## Write the Google sheet for this race

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Qualifying Women')

if qualifying_women.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(qualifying_women.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_women,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Qualifying Men')

if qualifying_men.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(qualifying_men.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_men,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Qualifying Nonbinary')

if qualifying_nonbinary.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(qualifying_nonbinary.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_nonbinary,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Top Women')

if top_women.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(top_women.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=top_women,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Top Men')

if top_men.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(top_men.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=top_men,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'Top Nonbinary')

if top_nonbinary.shape[0] > 0:
  ws = wb.worksheet(racefile_name)
  ws.add_rows(top_nonbinary.shape[0])
  gd.set_with_dataframe(worksheet=ws,dataframe=top_nonbinary,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

ws = wb.worksheet(racefile_name)
ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, 'All Team Results')

ws = wb.worksheet(racefile_name)
ws.add_rows(racedata.shape[0])
gd.set_with_dataframe(worksheet=ws,dataframe=racedata,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

In [291]:
## Append new qualifiers to the summary (with dates)

summary_data = sum_worksheet.get_all_values()

separator_row = ['****', '', '', '', '', '', '', '', '', '', '', '', '', '']
chunks = (list(g) for k,g in groupby(summary_data, key=lambda x: x != separator_row) if k)

leftover_chunks = []

for chunk in chunks:
  if chunk[0][0] == "Qualifying Women":
    qualifying_women_summary = chunk
  elif chunk[0][0] == "Qualifying Men":
    qualifying_men_summary = chunk
  elif chunk[0][0] == "Qualifying Nonbinary":
    qualifying_nonbinary_summary = chunk
  else:
    leftover_chunks += chunk  

## Build data frames from existing lists. This is probably unnecessary but I need
## to think through how to handle more stuff being added to the spreadsheet

qualifying_women_summary_df = pd.DataFrame(qualifying_women_summary)
qualifying_women_summary_df.columns = qualifying_women_summary_df.iloc[1]
qualifying_women_summary_df = qualifying_women_summary_df.iloc[2:]
qualifying_women_summary_df = qualifying_women_summary_df[qualifying_women_summary_df['Name'].str.strip().astype(bool)]

qualifying_men_summary_df = pd.DataFrame(qualifying_men_summary)
qualifying_men_summary_df.columns = qualifying_men_summary_df.iloc[1]
qualifying_men_summary_df = qualifying_men_summary_df.iloc[2:]
qualifying_men_summary_df = qualifying_men_summary_df[qualifying_men_summary_df['Name'].str.strip().astype(bool)]

qualifying_nonbinary_summary_df = pd.DataFrame(qualifying_nonbinary_summary)
qualifying_nonbinary_summary_df.columns = qualifying_nonbinary_summary_df.iloc[1]
qualifying_nonbinary_summary_df = qualifying_nonbinary_summary_df.iloc[2:]
qualifying_nonbinary_summary_df = qualifying_nonbinary_summary_df[qualifying_nonbinary_summary_df['Name'].str.strip().astype(bool)]

## Add new race data and then append new qualifiers

qualifying_women.insert(0, "Units", racedist_units)
qualifying_women.insert(0, "Distance", racedist)
qualifying_women.insert(0, "Date", racedate)
qualifying_women.insert(0, "Race", racefile_name.replace("_", " "))
qualifying_women_summary_df = pd.concat([qualifying_women_summary_df, qualifying_women])

qualifying_men.insert(0, "Units", racedist_units)
qualifying_men.insert(0, "Distance", racedist)
qualifying_men.insert(0, "Date", racedate)
qualifying_men.insert(0, "Race", racefile_name.replace("_", " "))
qualifying_men_summary_df = pd.concat([qualifying_men_summary_df, qualifying_men])

qualifying_nonbinary.insert(0, "Units", racedist_units)
qualifying_nonbinary.insert(0, "Distance", racedist)
qualifying_nonbinary.insert(0, "Date", racedate)
qualifying_nonbinary.insert(0, "Race", racefile_name.replace("_", " "))
qualifying_nonbinary_summary_df = pd.concat([qualifying_nonbinary_summary_df, qualifying_nonbinary])

In [292]:
## Rewrite the summary Google sheet with the new qualifiers appended

ws = wb.worksheet("Summary")
ws.clear()
ws.delete_rows(1, ws.row_count)

## Women list

ws = wb.worksheet("Summary")
ws.add_rows(2)
ws.update_cell(ws.row_count+1, 1, '****')
ws.update_cell(ws.row_count+2, 1, 'Qualifying Women')

ws = wb.worksheet("Summary")
ws.add_rows(qualifying_women_summary_df.shape[0]+2)
gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_women_summary_df,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

## Men list

ws = wb.worksheet("Summary")
ws.add_rows(2)
ws.update_cell(ws.row_count+1, 1, '****')
ws.update_cell(ws.row_count+2, 1, 'Qualifying Men')

ws = wb.worksheet("Summary")
ws.add_rows(qualifying_men_summary_df.shape[0]+2)
gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_men_summary_df,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

## Nonbinary list

ws = wb.worksheet("Summary")
ws.add_rows(2)
ws.update_cell(ws.row_count+1, 1, '****')
ws.update_cell(ws.row_count+2, 1, 'Qualifying Nonbinary')

ws = wb.worksheet("Summary")
ws.add_rows(qualifying_nonbinary_summary_df.shape[0]+2)
gd.set_with_dataframe(worksheet=ws,dataframe=qualifying_nonbinary_summary_df,include_index=False,include_column_header=True,row=ws.row_count+1,resize=False)

## Anything below the final **** (or strictly, anything in any other chunk will be re-appended here)

ws.add_rows(2)
ws.update_cell(ws.row_count+2, 1, '****')
wb.values_append("Summary", {'valueInputOption': 'USER_ENTERED'}, {'values': leftover_chunks})


{'spreadsheetId': '1n_xmaQuNmj3JBfOXJEN8c4HZAOUPOjtPAM-fY4h4K54',
 'tableRange': 'Summary!A12:N15',
 'updates': {'spreadsheetId': '1n_xmaQuNmj3JBfOXJEN8c4HZAOUPOjtPAM-fY4h4K54',
  'updatedRange': 'Summary!A16:N17',
  'updatedRows': 2,
  'updatedColumns': 14,
  'updatedCells': 28}}